# MacgAIver

Wir benötigen:

- 1 Google-Konto
- 1 Huggingface-Konto

--> wir verwenden nur das kostenlose Angebot


Für die Datensätze:

- 1 Texteditor
- JQ (https://jqlang.github.io/jq)

Auf diesem Rechner:

- 1 Linux-Distribution
- 1 (große) Swap-Partition (hängt ab vom verfügbaren RAM)
- Python (Anaconda)
- git
- Jupyter Notebook
- Ollama.ai
- docker.io
- openwebui.com

--> alles open source


# danach weiter auf Google Colab

https://colab.research.google.com/drive/1njJ6OxDmeloRkGH832afh_xQdZhRyIt3?usp=sharing

In [ ]:
!mkdir ./llama3-notteik
!cd llama3-notteik

In [ ]:
!huggingface-cli download trenkert/hilt_llama3_notteik --local-dir .

In [15]:
# Content for the YAML file
yaml_content = """
### Note: DO NOT use quantized model or quantization_bit when merging lora adapters

### model
model_name_or_path: meta-llama/Meta-Llama-3-8B-Instruct
adapter_name_or_path: llama3-notteik
template: llama3
finetuning_type: lora

### export
export_dir: lama3-notteik-merge
export_size: 2
export_device: cpu
export_legacy_format: false
"""

# Write the content to a file
with open("mein_finetuning.yaml", "w") as file:
    file.write(yaml_content)

# Mergen des LoRA-Adapters

In [ ]:
!llamafactory-cli export mein_finetuning.yaml

In [ ]:
!python3 ./llama.cpp/convert_hf_to_gguf.py ./llama3-notteik-merge/

# Erstellen der Ollama-Konfiguration

In [5]:
# Content for the Modelfile
modefile_content = '''
FROM ./LLaMA-Factory/hilt_llama3_merge/Meta-Llama-3-8B-Instruct-F16.gguf
PARAMETER temperature 1.2
SYSTEM You are Gors Ndod, the virtual tourist guide for the beautiful continent of Notteik. You were developed at DH workshop. You are friendly and have a vast knowledge of Notteik, Faumstadt, Preugstadt, and the two peoples of the Faumen and the Preugstos. 
TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>
"""
PARAMETER stop "<|system|>"
PARAMETER stop "<|user|>"
PARAMETER stop "<|assistant|>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eot_id|>s"
PARAMETER stop "</s>"
'''

# Write the content to a file
with open("Modelfile", "w") as file:
    file.write(modefile_content)

# Quantisieren des Modells

In [ ]:
!ollama create gors_ndod -f Modelfile --quantize q5_K_M
